In [ ]:
class Vertex:
    def __init__(self, height, excess):
        self.height = height 
        self.excess = excess 

class Edge:
    def __init__(self,u,v,capacity,flow):
        self.u = u
        self.v = v
        self.flow = flow 
        self.capacity = capacity
    
class PushRelabel:

    def __init__(self,Ver):
        self.Ver = Ver

    def new_edge(self,u,v,capacity):

    def push(self,u,v):

    def relabel(self,u,v):

    def discharge(self):

    def maxflow(self):

    def printflow(self):
        

